# Stratified Kfold Cross Validation

- class sklearn.model_selection.StratifiedKFold(n_splits=5, *, shuffle=False, random_state=None)

> Parameters
- n_splitsint, default=5
    Number of folds. Must be at least 2.

    Changed in version 0.22: n_splits default value changed from 3 to 5.

    shufflebool, default=False
    Whether to shuffle each class’s samples before splitting into batches. Note that the samples within each split will not be shuffled.

    random_stateint, RandomState instance or None, default=None
    When shuffle is True, random_state affects the ordering of the indices, which controls the randomness of each fold for each class. Otherwise, leave random_state as None. Pass an int for reproducible output across multiple function calls. See Glossary.

In [1]:
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

import pandas as pd
import numpy as np

In [6]:
from sklearn.model_selection import KFold

In [3]:
from sklearn.datasets import load_iris

### examples

In [4]:
iris = load_iris()

iris_df = pd.DataFrame(data = iris.data, columns = iris.feature_names)
iris_df['label'] = iris.target
iris_df['label'].value_counts()

0    50
1    50
2    50
Name: label, dtype: int64

In [7]:
# K Fold (비교용  w/ Stratified KFold)

kfold = KFold(n_splits = 3)

# kfold.split(X)는 폴드 세트를 3번 반복할 떄마다 달라지는 학습/테스트 용 데이터 로우 인덱스 번호 반환

n_iter = 0
for train_index, test_index in kfold.split(iris_df) :
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]

    print('##교차 검증: {0}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts())

##교차 검증: 1
학습 레이블 데이터 분포:
 1    50
2    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    50
Name: label, dtype: int64
##교차 검증: 2
학습 레이블 데이터 분포:
 0    50
2    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    50
Name: label, dtype: int64
##교차 검증: 3
학습 레이블 데이터 분포:
 0    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    50
Name: label, dtype: int64


In [8]:
# Stratified KFold

skf = StratifiedKFold(n_splits=3)
n_iter= 0

for train_index, test_index in skf.split(iris_df, iris_df['label']) :
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]

    print('## 교차 검증: {0}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts())

## 교차 검증: 1
학습 레이블 데이터 분포:
 2    34
0    33
1    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    17
1    17
2    16
Name: label, dtype: int64
## 교차 검증: 2
학습 레이블 데이터 분포:
 1    34
0    33
2    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    17
2    17
1    16
Name: label, dtype: int64
## 교차 검증: 3
학습 레이블 데이터 분포:
 0    34
1    33
2    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    17
2    17
0    16
Name: label, dtype: int64


In [11]:
# 정확도 평가 : Stratified K Fold

dt_clf = DecisionTreeClassifier(random_state=11)

features = iris.data
label = iris.target

skfold = StratifiedKFold(n_splits=3)
n_iter = 0
cv_accuracy = []

# StratifiedKFold의 split() 호출시 반드시 레이블 데이터 셋도 추가 입력 필요 -> 비율 맞추는 기준 
for train_index, test_index in skfold.split(features, label) :
    # split()으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]

    #학습 및 예측
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)

    #반복 시 마다 정확도 측정
    n_iter += 1
    accuracy = np.around(accuracy_score(y_test,pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]

    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기:{3}'.format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스:{1}'.format(n_iter,test_index))
    cv_accuracy.append(accuracy)

# 교차 검증별 정확도 및 평균 정확도 계산
print('\n## 교차 검증별 정확도', np.round(cv_accuracy, 4))
print('## 평균 검증 정확도', np.mean(cv_accuracy))


#1 교차 검증 정확도 :0.98, 학습 데이터 크기: 100, 검증 데이터 크기:50
#1 검증 세트 인덱스:[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115]

#2 교차 검증 정확도 :0.92, 학습 데이터 크기: 100, 검증 데이터 크기:50
#2 검증 세트 인덱스:[ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132]

#3 교차 검증 정확도 :0.98, 학습 데이터 크기: 100, 검증 데이터 크기:50
#3 검증 세트 인덱스:[ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]

## 교차 검증별 정확도 [0.98 0.92 0.98]
## 평균 검증 정확도 0.96


In [12]:
# 정확도 평가 : KFold

dt_clf = DecisionTreeClassifier(random_state=11)

features = iris.data
label = iris.target

kfold = KFold(n_splits=3)
n_iter = 0
cv_accuracy = []

# StratifiedKFold의 split() 호출시 반드시 레이블 데이터 셋도 추가 입력 필요 -> 비율 맞추는 기준 
for train_index, test_index in kfold.split(features, label) :
    # split()으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]

    #학습 및 예측
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)

    #반복 시 마다 정확도 측정
    n_iter += 1
    accuracy = np.around(accuracy_score(y_test,pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]

    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기:{3}'.format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스:{1}'.format(n_iter,test_index))
    cv_accuracy.append(accuracy)

# 교차 검증별 정확도 및 평균 정확도 계산
print('\n## 교차 검증별 정확도', np.round(cv_accuracy, 4))
print('## 평균 검증 정확도', np.mean(cv_accuracy))


#1 교차 검증 정확도 :0.0, 학습 데이터 크기: 100, 검증 데이터 크기:50
#1 검증 세트 인덱스:[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49]

#2 교차 검증 정확도 :0.0, 학습 데이터 크기: 100, 검증 데이터 크기:50
#2 검증 세트 인덱스:[50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73
 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97
 98 99]

#3 교차 검증 정확도 :0.0, 학습 데이터 크기: 100, 검증 데이터 크기:50
#3 검증 세트 인덱스:[100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117
 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]

## 교차 검증별 정확도 [0. 0. 0.]
## 평균 검증 정확도 0.0


# cross_val_score

> 교차 검증 간편.ver
- for문 대신 cross_val_score 한 줄로 가능 


sklearn.model_selection.cross_val_score(estimator, X, y=None, *, groups=None, scoring=None, cv=None, n_jobs=None, verbose=0, fit_params=None, pre_dispatch='2*n_jobs', error_score=nan)

### Parameters

- estimatorestimator object implementing ‘fit’
The object to use to fit the data.

- X : array-like of shape (n_samples, n_features)
    The data to fit. Can be for example a list, or an array.

- y : array-like of shape (n_samples,) or (n_samples, n_outputs), default=None
    The target variable to try to predict in the case of supervised learning.

- groups : array-like of shape (n_samples,), default=None
    Group labels for the samples used while splitting the dataset into train/test set. Only used in conjunction with a “Group” cv instance (e.g., GroupKFold).

- scoring : str or callable, default=None
    A str (see model evaluation documentation) or a scorer callable object / function with signature scorer(estimator, X, y) which should return only a single value.

    Similar to cross_validate but only a single metric is permitted.

    If None, the estimator’s default scorer (if available) is used.

- cv : int, cross-validation generator or an iterable, default=None
    - 이 자체가 stratified Kfold 적용
    - 폴드(중첩) 수
    - For int/None inputs, if the estimator is a classifier and y is either binary or multiclass, StratifiedKFold is used. In all other cases, KFold is used. These splitters are instantiated with shuffle=False so the splits will be the same across calls.


- n_jobs: int, default=None
    Number of jobs to run in parallel. Training the estimator and computing the score are parallelized over the cross-validation splits. None means 1 unless in a joblib.parallel_backend context. -1 means using all processors. See Glossary for more details.

- verbose : int, default=0
    The verbosity level.

- fit_params : dict, default=None
    Parameters to pass to the fit method of the estimator.

- pre_dispatch : int or str, default=’2*n_jobs’
    Controls the number of jobs that get dispatched during parallel execution. Reducing this number can be useful to avoid an explosion of memory consumption when more jobs get dispatched than CPUs can process. This parameter can be:

    - None, in which case all the jobs are immediately created and spawned. Use this for lightweight and fast-running jobs, to avoid delays due to on-demand spawning   of the jobs

    - An int, giving the exact number of total jobs that are spawned

    - A str, giving an expression as a function of n_jobs, as in ‘2*n_jobs’

- error_score : ‘raise’ or numeric, default=np.nan
    Value to assign to the score if an error occurs in estimator fitting. If set to ‘raise’, the error is raised. If a numeric value is given, FitFailedWarning is raised.

In [13]:
from sklearn.model_selection import cross_val_score, cross_validate

In [14]:
iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)

data = iris_data.data
label = iris_data.target

# 성능 지표는 정확도(accuracy), 교차검증세트 5개
scores = cross_val_score(dt_clf, data, label, scoring='accuracy', cv=5)

print('교차 검증별 정확도:', np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

교차 검증별 정확도: [0.9667 0.9667 0.9    0.9667 1.    ]
평균 검증 정확도: 0.96


# GridSearchCV
> sklearn.model_selection.GridSearchCV

- class sklearn.model_selection.GridSearchCV(estimator, param_grid, *, scoring=None, n_jobs=None, refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', error_score=nan, return_train_score=False)


### Parameters

- estimator : estimator object

- param_grid : dict or list of dictionaries
    - grid를 parameter로 사용
    - grid에 뭘 넣을 지 입력해야

- scoring : str, callable, list, tuple or dict, default=None

- n_jobs : int, default=None

- refit : bool, str, or callable, default=True
    - Refit an estimator using the best found parameters on the whole dataset.
    - 최적의 값을 가졌다면, 그 값을 유지
- cv : int, cross-validation generator or an iterable, default=None

- verbose : int

- pre_dispatch : int, or str, default=’2*n_jobs’

- error_score : ‘raise’ or numeric, default=np.nan

- return_train_score : bool, default=False


In [31]:
from sklearn.datasets import load_iris
from sklearn.model_selection import GridSearchCV, train_test_split


In [32]:
from sklearn.tree import DecisionTreeClassifier

In [41]:
# 데이터 로딩하고 학습데이터와 데스트데이터 분리
iris= load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size = 0.2, random_state = 121)

dtree = DecisionTreeClassifier(random_state=11)

# parameter
parameters = {'max_depth' : [1,2,3], 'min_samples_split' : [2, 3] }




In [42]:
# param_grid의 하이퍼 파라미터들을 3개의 train, test set fold로 나누어서 테스트 수행 설정 
### refit = True가 default. True이면 가장 좋은 파라미터 설정으로 재학습 시킴

grid_dtree = GridSearchCV(dtree, param_grid = parameters, cv = 3, refit = True)

# iris Train 데이터로 param_grid의 하이퍼 파라미터들을 순차적으로 학습/평가.
grid_dtree.fit(X_train, y_train)

# GridSearchCV 결과 추추라여 DataFrame으로 변환
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', 'split0_test_score', 'split1_test_score', 'split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


In [43]:
grid_dtree.best_estimator_

DecisionTreeClassifier(max_depth=3, random_state=11)

In [44]:
grid_dtree.best_params_

{'max_depth': 3, 'min_samples_split': 2}

In [45]:
X = np.array([5.2, 3.3, 1.2, 0.3]).reshape(1,-1)
grid_dtree.predict(X)

array([0])

In [46]:
iris.data[0]

array([5.1, 3.5, 1.4, 0.2])

In [47]:
iris.target[0]

0